In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
from keras import layers, models
import numpy as np
import capsnet
import capsule_layers
import dataset
import model_functions as mf
import audio_functions as af

import IPython.display as ipd
import matplotlib.pyplot as plt


/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
Using TensorFlow backend.


In [12]:
#  Set other variables
sample_rate=16384
n_fft=1024
fft_hop=256
patch_window=256
patch_hop=128
n_parallel_readers=2
normalise=True
batch_size = 5
shuffle=False
n_shuffle = 10

#directory_a = 'C:/Users/Toby/MSc_Project/Test_Audio/CHiME/test/Mixed'
#directory_b = 'C:/Users/Toby/MSc_Project/Test_Audio/CHiME/test/Voice'

directory_a = '/home/enterprise.internal.city.ac.uk/acvn728/LibriSpeechMini/Voice/train-clean-100/19/198/'
directory_b = '/home/enterprise.internal.city.ac.uk/acvn728/LibriSpeechMini/Mixed/train-clean-100/19/198/'

#  Create the pipeline
tf.reset_default_graph()
data = dataset.zip_files(directory_a, directory_b)
data = dataset.get_paired_dataset(data,
                                  sample_rate,
                                  n_fft,
                                  fft_hop,
                                  patch_window,
                                  patch_hop,
                                  n_parallel_readers,
                                  batch_size,
                                  n_shuffle,
                                  normalise)

#  Create the iterator
mixed_spec, voice_spec, mixed_audio, voice_audio = data.make_one_shot_iterator().get_next()

#  Create variable placeholders
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_mag = tf.expand_dims(mixed_spec[:, :, :-1, 0], 3)
mixed_phase = tf.expand_dims(mixed_spec[:, :, :-1, 1], 3)
voice_mag = tf.expand_dims(voice_spec[:, :, :-1, 0], 3)

In [3]:
mixed_mag.get_shape()

TensorShape([Dimension(None), Dimension(256), Dimension(512), Dimension(1)])

In [4]:
class SegCaps_CapsNetBasic(object):
    
    def __init__(self, mixed_mag, mixed_phase, voice_mag, is_training, reuse=True, name='SegCaps_CapsNetBasic'):
        """
        input_tensor: Tensor with shape [batch_size, height, width, channels]
        is_training:  Boolean - should the model be trained on the current input or not
        name:         Model instance name
        """
        with tf.variable_scope(name):
            self.mixed_mag = mixed_mag
            self.mixed_phase = mixed_phase
            self.voice_mag = voice_mag
            
            with tf.variable_scope('Convolution'):
                conv1 = mf.conv(mixed_mag, filters=128, kernel_size=5, stride=(1, 1))
                
                # Reshape layer to be 1 capsule x [filters] atoms
                _, H, W, C = conv1.get_shape()
                conv1 = layers.Reshape((H.value, W.value, 1, C.value))(conv1)
                #conv1 = tf.expand_dims(conv1, 2)
                self.conv1 = conv1
            
            with tf.variable_scope('Primary_Caps'):
                primary_caps = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=8, num_atoms=16, strides=1, padding='same',
                                                               routings=1, name='primarycaps')(conv1)
                self.primary_caps = primary_caps
                
            with tf.variable_scope('Seg_Caps'):
                seg_caps = capsule_layers.ConvCapsuleLayer(kernel_size=1, num_capsule=1, num_atoms=16, strides=1, padding='same',
                                                           routings=3, name='seg_caps')(primary_caps)
                self.seg_caps = seg_caps
            
            with tf.variable_scope('Reconstruction'):
                reconstruction = capsule_layers.ConvCapsuleLayer(kernel_size=1, num_capsule=1, num_atoms=1, strides=1, padding='same',
                                                           routings=3, name='seg_caps')(seg_caps)
                reconstruction = tf.squeeze(reconstruction,-1)
                self.reconstruction = reconstruction
            
            self.cost = mf.l1_loss(self.reconstruction, voice_mag)

            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=0.0002,
                beta1=0.5,
            )
            self.train_op = self.optimizer.minimize(self.cost)

In [5]:
class UNetModel(object):
    """
    Top level U-Net object.
    Attributes:
        mixed_mag: Input placeholder for magnitude spectrogram of mixed signals (voice plus background noise) - X
        voice_mag: Input placeholder for magnitude spectrogram of isolated voice signal - Y
        mixed_phase: Input placeholder for phase spectrogram of mixed signals (voice plus background noise)
        mixed_audio: Input placeholder for waveform audio of mixed signals (voice plus background noise)
        voice_audio: Input placeholder for waveform audio of isolated voice signal
        variant: The type of U-Net model (Normal convolutional or capsule based)
        is_training: Boolean - should the model be trained on the current input or not
        name: Model instance name
    """
    def __init__(self, mixed_mag, voice_mag, mixed_phase, mixed_audio, voice_audio, variant, is_training, name):
        with tf.variable_scope(name):
            self.mixed_mag = mixed_mag
            self.voice_mag = voice_mag
            self.mixed_phase = mixed_phase
            self.mixed_audio = mixed_audio
            self.voice_audio = voice_audio
            self.variant = variant
            self.is_training = is_training

            self.voice_mask_unet = UNet(mixed_mag, variant, is_training=is_training, reuse=False, name='voice-mask-unet')

            self.voice_mask = self.voice_mask_unet.output

            self.gen_voice = self.voice_mask * mixed_mag

            self.pw_cost = mf.pw_l1_loss(self.gen_voice, voice_mag)
            self.cost = tf.reduce_mean(self.pw_cost)

            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=0.0002,
                beta1=0.5,
            )
            self.train_op = self.optimizer.minimize(self.pw_cost)


class UNet(object):

    def __init__(self, input_tensor, variant, is_training, reuse, name):
        with tf.variable_scope(name, reuse=reuse):
            self.variant = variant

            if self.variant == 'unet':
                self.encoder = UNetEncoder(input_tensor, is_training, reuse)
                self.decoder = UNetDecoder(self.encoder.output, self.encoder, is_training, reuse)
            elif self.variant == 'capsunet':
                self.encoder = CapsUNetEncoder(input_tensor, is_training, reuse)
                self.decoder = CapsUNetDecoder(self.encoder.output, self.encoder, is_training, reuse)

            self.output = mf.tanh(self.decoder.output) / 2 + .5

class UNetEncoder(object):
    """
    The down-convolution side of a convoltional U-Net model.
    """

    def __init__(self, input_tensor, is_training, reuse):
        net = input_tensor
        with tf.variable_scope('encoder'):
            with tf.variable_scope('layer-1'):
                net = mf.conv(net, filters=16, kernel_size=5, stride=(2, 2))
                self.l1 = net

            with tf.variable_scope('layer-2'):
                net = mf.lrelu(net)
                net = mf.conv(net, filters=32, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                self.l2 = net

            with tf.variable_scope('layer-3'):
                net = mf.lrelu(net)
                net = mf.conv(net, filters=64, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                self.l3 = net

            with tf.variable_scope('layer-4'):
                net = mf.lrelu(net)
                net = mf.conv(net, filters=128, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                self.l4 = net

            with tf.variable_scope('layer-5'):
                net = mf.lrelu(net)
                net = mf.conv(net, filters=256, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                self.l5 = net

            with tf.variable_scope('layer-6'):
                net = mf.lrelu(net)
                net = mf.conv(net, filters=512, kernel_size=5, stride=(2, 2))

            self.output = net


class UNetDecoder(object):
    """
    The up-convolution side of a convolutional U-Net model
    """
    def __init__(self, input_tensor, encoder, is_training, reuse):
        net = input_tensor

        with tf.variable_scope('decoder'):
            with tf.variable_scope('layer-1'):
                net = mf.relu(net)
                net = mf.deconv(net, filters=256, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                net = mf.dropout(net, .5)

            with tf.variable_scope('layer-2'):
                net = mf.relu(mf.concat(net, encoder.l5))
                net = mf.deconv(net, filters=128, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                net = mf.dropout(net, .5)

            with tf.variable_scope('layer-3'):
                net = mf.relu(mf.concat(net, encoder.l4))
                net = mf.deconv(net, filters=64, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)
                net = mf.dropout(net, .5)

            with tf.variable_scope('layer-4'):
                net = mf.relu(mf.concat(net, encoder.l3))
                net = mf.deconv(net, filters=32, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)

            with tf.variable_scope('layer-5'):
                net = mf.relu(mf.concat(net, encoder.l2))
                net = mf.deconv(net, filters=16, kernel_size=5, stride=(2, 2))
                net = mf.batch_norm(net, is_training=is_training, reuse=reuse)

            with tf.variable_scope('layer-6'):
                net = mf.relu(mf.concat(net, encoder.l1))
                net = mf.deconv(net, filters=1, kernel_size=5, stride=(2, 2))

            self.output = net


class CapsUNetEncoder(object):
    """
    The down-convolutional side of a capsule based U-Net model (based on SegCaps R3 model).
    """
    def __init__(self, input_tensor, is_training, reuse):
        #net = layers.Input(shape=input_tensor)
        net = input_tensor
        with tf.variable_scope('Encoder'):
            with tf.variable_scope('Convolution'):
                # Layer 1: Just a conventional Conv2D layer
                net = layers.Conv2D(filters=16, kernel_size=5, strides=1, padding='same', activation='relu', name='conv1')(net)

                # Reshape layer to be 1 capsule x [filters] atoms
                _, H, W, C = net.get_shape()
                net = layers.Reshape((H.value, W.value, 1, C.value))(net)
                self.conv1 = net

            with tf.variable_scope('Primary_Caps'):
                # Layer 1: Primary Capsule: Conv cap with routing 1
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=2, num_atoms=16, strides=2, padding='same',
                                                    routings=1, name='primarycaps')(net)
                self.primary_caps = net

            with tf.variable_scope('Conv_caps_2'):
                # Layer 2: Convolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=4, num_atoms=16, strides=1, padding='same',
                                                      routings=3, name='conv_cap_2_1')(net)
                self.conv_cap_2_1 = net

                # Layer 2: Convolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=4, num_atoms=32, strides=2, padding='same',
                                                      routings=3, name='conv_cap_2_2')(net)
                self.conv_cap_2_2 = net

            with tf.variable_scope('Conv_caps_3'):
                # Layer 3: Convolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=8, num_atoms=32, strides=1, padding='same',
                                                      routings=3, name='conv_cap_3_1')(net)
                self.conv_cap_3_1 = net

                # Layer 3: Convolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=8, num_atoms=64, strides=2, padding='same',
                                                    routings=3, name='conv_cap_3_2')(net)
                self.conv_cap_3_2 = net

            with tf.variable_scope('Conv_caps_4'):
                # Layer 4: Convolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=8, num_atoms=32, strides=1, padding='same',
                                                      routings=3, name='conv_cap_4_1')(net)

            self.output = net


class CapsUNetDecoder(object):
    """
    The up-convolutional side of a capsule based U-Net model.
    """
    def __init__(self, input_tensor, encoder, is_training, reuse):
        net = input_tensor
        with tf.variable_scope('Decoder'):
            with tf.variable_scope('UpCaps_1'):
                    # Layer 1 Up: Deconvolutional Capsule
                net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=8, num_atoms=32, upsamp_type='deconv',
                                                        scaling=2, padding='same', routings=3, name='deconv_cap_1_1')(net)
                self.upcap_1_1 = net

                # Skip connection
                net = layers.Concatenate(axis=-2, name='up_1')([net, encoder.conv_cap_3_1])

                # Layer 1 Up: Deconvolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=4, num_atoms=32, strides=1,
                                                      padding='same', routings=3, name='deconv_cap_1_2')(net)
                self.upcap_1_2 = net

            with tf.variable_scope('UpCaps_2'):
                # Layer 2 Up: Deconvolutional Capsule
                net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=4, num_atoms=16, upsamp_type='deconv',
                                                        scaling=2, padding='same', routings=3, name='deconv_cap_2_1')(net)
                self.upcap_2_1 = net

                # Skip connection
                net = layers.Concatenate(axis=-2, name='up_2')([net, encoder.conv_cap_2_1])

                # Layer 2 Up: Deconvolutional Capsule
                net = capsule_layers.ConvCapsuleLayer(kernel_size=5, num_capsule=4, num_atoms=16, strides=1,
                                                      padding='same', routings=3, name='deconv_cap_2_2')(net)
                self.upcap_2_2 = net

            with tf.variable_scope('UpCaps_3'):
                # Layer 3 Up: Deconvolutional Capsule
                net = capsule_layers.DeconvCapsuleLayer(kernel_size=4, num_capsule=2, num_atoms=16, upsamp_type='deconv',
                                                        scaling=2, padding='same', routings=3, name='deconv_cap_3_1')(net)
                self.upcap_3_1 = net

                # Skip connection
                net = layers.Concatenate(axis=-2, name='up_3')([net, encoder.conv1])
            with tf.variable_scope('Reconstruction'):
                # Layer 4: Convolutional Capsule: 1x1
                net = capsule_layers.ConvCapsuleLayer(kernel_size=1, num_capsule=1, num_atoms=16, strides=1, padding='same',
                                                      routings=3, name='seg_caps')(net)
                
                _, H, W, C, D = net.get_shape()
                
                net = layers.Reshape((H.value, W.value, D.value))(net)

                net = layers.Conv2D(filters=64, kernel_size=1, padding='same', kernel_initializer='he_normal',
                                    activation='relu', name='recon_1')(net)

                net = layers.Conv2D(filters=128, kernel_size=1, padding='same', kernel_initializer='he_normal',
                                    activation='relu', name='recon_2')(net)

                net = layers.Conv2D(filters=1, kernel_size=1, padding='same', kernel_initializer='he_normal',
                                    activation='sigmoid', name='out_recon')(net)

            self.output = net

In [13]:
#tf.reset_default_graph()
#model = SegCaps_CapsNetBasic(mixed_mag, mixed_phase, voice_mag, is_training=False)
model = UNetModel(mixed_mag, voice_mag, mixed_phase, mixed_audio, voice_audio, 'capsunet', is_training=True, name='R3_CapsNet')
#model = UNetModel(mixed_mag, voice_mag, mixed_phase, mixed_audio, voice_audio, 'unet', is_training=True, name='R3_CapsNet')

### Basic Caps Net - Layer Dimensions

In [11]:
print('Layers\' Shapes:\n'
      '\nInput: ', mixed_mag.get_shape().as_list(),
      '\nConvolution: ',model.conv1.get_shape().as_list(),
      '\nPrimary Caps: ',model.primary_caps.get_shape().as_list(),
      '\nSeg Caps: ',model.seg_caps.get_shape().as_list(),
      '\nRecontruction: ',model.reconstruction.get_shape().as_list())

Layers' Shapes:

Input:  [None, 256, 512, 1] 
Convolution:  [None, 256, 512, 1, 128] 
Primary Caps:  [None, 256, 512, 8, 16] 
Seg Caps:  [None, 256, 512, 1, 16] 
Recontruction:  [None, 256, 512, 1]


### R3 Caps Net - Layer Dimensions

In [8]:
print('Layers\' Shapes:\n'
      '\nInput: ', mixed_mag.get_shape().as_list(),
      '\nConvolution: ',model.voice_mask_unet.encoder.conv1.get_shape().as_list(),
      '\nPrimary Caps: ',model.voice_mask_unet.encoder.primary_caps.get_shape().as_list(),
      '\nConv Caps 2.1: ',model.voice_mask_unet.encoder.conv_cap_2_1.get_shape().as_list(),
      '\nConv Caps 2.2: ',model.voice_mask_unet.encoder.conv_cap_2_2.get_shape().as_list(),
      '\nConv Caps 3.1: ',model.voice_mask_unet.encoder.conv_cap_3_1.get_shape().as_list(),
      '\nConv Caps 3.2: ',model.voice_mask_unet.encoder.conv_cap_3_2.get_shape().as_list(),
      '\nConv Caps 4 (output): ',model.voice_mask_unet.encoder.output.get_shape().as_list(),
      
      '\n\nUp Caps 1.1: ',model.voice_mask_unet.decoder.upcap_1_1.get_shape().as_list(),
      '\nUp Caps 1.2: ',model.voice_mask_unet.decoder.upcap_1_2.get_shape().as_list(),
      '\nUp Caps 2.1: ',model.voice_mask_unet.decoder.upcap_2_1.get_shape().as_list(),
      '\nUp Caps 2.2: ',model.voice_mask_unet.decoder.upcap_2_2.get_shape().as_list(),
      '\nUp Caps 3.1: ',model.voice_mask_unet.decoder.upcap_3_1.get_shape().as_list(),
      '\nUp Caps 4 (output): ',model.voice_mask_unet.decoder.output.get_shape().as_list()
      )

Layers' Shapes:

Input:  [None, 256, 512, 1] 
Convolution:  [None, 256, 512, 1, 16] 
Primary Caps:  [None, 128, 256, 2, 16] 
Conv Caps 2.1:  [None, 128, 256, 4, 16] 
Conv Caps 2.2:  [None, 64, 128, 4, 32] 
Conv Caps 3.1:  [None, 64, 128, 8, 32] 
Conv Caps 3.2:  [None, 32, 64, 8, 64] 
Conv Caps 4 (output):  [None, 32, 64, 8, 32] 

Up Caps 1.1:  [None, 64, 128, 8, 32] 
Up Caps 1.2:  [None, 64, 128, 4, 32] 
Up Caps 2.1:  [None, 128, 256, 4, 16] 
Up Caps 2.2:  [None, 128, 256, 4, 16] 
Up Caps 3.1:  [None, 256, 512, 2, 16] 
Up Caps 4 (output):  [None, 256, 512, 1]


In [ ]:
mixed_mag

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
mix, phase, voice, mask = sess.run([model.mixed_mag, model.mixed_phase, model.voice_mag, model.reconstruction])

In [ ]:
plt.rcParams['image.cmap'] = 'hot'

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20,10))

ax[0,0].imshow(mix[0,:,:,0])
ax[0,0].set_title('mixed input')
ax[0,1].imshow(voice[0,:,:,0])
ax[0,1].set_title('isolated voice')
ax[1,0].imshow(mask[0,:,:,0])
ax[1,0].set_title('mask')
ax[1,1].imshow(mix[0,:,:,0] * mask[0,:,:,0])
ax[1,1].set_title('generated voice')
plt.show()

In [ ]:
for i in range(1):
    _, mix, phase, voice, mask, cost = sess.run([model.train_op, model.mixed_mag, model.mixed_phase, 
                                                 model.voice_mag, model.voice_mask, model.cost])
    
    print(cost)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(20,10))

ax[0,0].imshow(mix[0,:,:,0])
ax[0,0].set_title('mixed input')
ax[0,1].imshow(voice[0,:,:,0])
ax[0,1].set_title('isolated voice')
ax[1,0].imshow(mask[0,:,:,0])
ax[1,0].set_title('mask')
ax[1,1].imshow(mix[0,:,:,0] * mask[0,:,:,0])
ax[1,1].set_title('generated voice')
plt.show()

In [ ]:
mix_wave = []
voice_wave = []
gen_wave = []

for i in range(mix.shape[0]):
    mix_wave.append(af.spectrogramToAudioFile(np.squeeze(mix[i, :, :, :]).T, n_fft,
                                                    fft_hop, phase=np.squeeze(phase[i, :, :, :]).T))
    voice_wave.append(af.spectrogramToAudioFile(np.squeeze(voice[i, :, :, :]).T, n_fft,
                                                    fft_hop, phase=np.squeeze(phase[i, :, :, :]).T))
    gen_wave.append(af.spectrogramToAudioFile(np.squeeze(mix[i, :, :, :] * mask[i, :, :, :]).T, n_fft,
                                                    fft_hop, phase=np.squeeze(phase[i, :, :, :]).T))

In [ ]:
#  The original mixture
ipd.Audio(mix_wave[0], rate=sample_rate)

In [ ]:
#  The isolated voice
ipd.Audio(voice_wave[0], rate=sample_rate)

In [ ]:
#  The model output
ipd.Audio(gen_wave[0], rate=sample_rate)

In [ ]:
plt.imshow(mask[4,:,:,0])
plt.show()

In [ ]:
mix[0,:,:,0] * mask[0,:,:,0]

In [ ]:
voice[0,:,:,0]